In [ ]:
import numpy as np
import scipy
import scipy.io as sio
import ahrs
from ahrs.filters import EKF
from ahrs.common.orientation import acc2q
import pandas as pd


In [ ]:
acc = pd.read_csv('./sensor_data/acc.csv')
gyro = pd.read_csv('./sensor_data/ang_vel.csv')
mag = pd.read_csv('./sensor_data/mag.csv')



In [ ]:
print(acc.shape, gyro.shape, mag.shape)
num_samples = min(acc.shape[0], gyro.shape[0], mag.shape[0])
num_samples

In [ ]:
# drop timestamp column
acc.drop(['Timestamp'], axis=1, inplace=True)
gyro.drop(['Timestamp'], axis=1, inplace=True)
mag.drop(['Timestamp'], axis=1, inplace=True)

acc.drop(acc.index[num_samples:], inplace=True)
gyro.drop(gyro.index[num_samples:], inplace=True)
mag.drop(mag.index[num_samples:], inplace=True)

print(acc.shape, gyro.shape, mag.shape)

assert acc.shape == gyro.shape == mag.shape


In [ ]:
acc_list = acc.to_numpy()
gyro_list = gyro.to_numpy()
mag_list = mag.to_numpy()

In [ ]:
ekf = EKF(gyr=gyro_list, acc=acc_list, mag=mag_list)
ekf.Q

In [ ]:
# from ahrs.common import to_angles
from ahrs.common.quaternion import QuaternionArray
Q = QuaternionArray(ekf.Q)
Q.to_angles()

# array for quaternions
# each index represents estimated quaternion for each sensor measurement  
Q = np.zeros((num_samples, 4))
# set initial position from acceleration alonr
Q[0] = acc2q(acc_list[0])
for t in range(1, num_samples):
	Q[t] = ekf.update(Q[t-1], gyro_list[t], acc_list[t])